# 🎮 Steam Gaming Analytics: Product Portfolio Intelligence

## Project Overview
**Business Questions:**
1. What pricing models drive the highest market penetration?
2. How do successful games position themselves competitively?

**Data Sources:**
- SteamSpy API: Player counts & market penetration data
- Steam Store API: Official pricing, genres, and positioning data
- Target: Multi-source analytics pipeline with BigQuery

---

## 📦 1. Data Collection & Multi-Source Integration

### 1.1 Setup & Authentication

In [ ]:
# Steam Gaming Analytics - Multi-Source Data Pipeline
# Portfolio Project: Product Portfolio & Market Intelligence

# Install required libraries
!pip install google-cloud-bigquery google-cloud-storage requests pandas matplotlib seaborn

# Import libraries
import pandas as pd
import requests
import json
from google.colab import auth
from google.cloud import bigquery
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import time

print("📦 Libraries installed successfully!")

📦 Libraries installed successfully!


In [ ]:
# Authenticate with Google Cloud
auth.authenticate_user()

# Set your GCP project ID
PROJECT_ID = "steam-analytics-portfolio"  #Project ID on GCP

# Initialize BigQuery client
client = bigquery.Client(project=PROJECT_ID)

print(f"✅ Successfully authenticated!")
print(f"🎯 Project ID: {PROJECT_ID}")
print(f"📊 BigQuery client initialized")



✅ Successfully authenticated!
🎯 Project ID: steam-analytics-portfolio
📊 BigQuery client initialized


In [ ]:
# Multi-Source Data Collection - Source 1: SteamSpy API

def get_steamspy_detailed_data(max_games=100, delay=1.0):
    """
    Get detailed data from SteamSpy API using appdetails endpoint
    This gives us rich engagement data instead of basic list data
    """
    print(f"🎯 Collecting detailed SteamSpy data for top {max_games} games...")

    # First, get the list of trending games
    base_url = "https://steamspy.com/api.php"
    list_params = {"request": "top100in2weeks"}

    try:
        response = requests.get(base_url, params=list_params)
        response.raise_for_status()
        game_list = response.json()
        print(f"✅ Retrieved list of {len(game_list)} top games")
    except Exception as e:
        print(f"❌ Error getting game list: {e}")
        return None

    # Now get detailed data for each game
    detailed_data = {}
    app_ids = list(game_list.keys())[:max_games]

    print(f"📊 Fetching detailed data for {len(app_ids)} games...")

    for i, app_id in enumerate(app_ids):
        try:
            detail_params = {
                "request": "appdetails",
                "appid": app_id
            }

            response = requests.get(base_url, params=detail_params)
            response.raise_for_status()
            game_data = response.json()

            if game_data and game_data.get('name'):
                detailed_data[app_id] = game_data
                print(f"  ✅ {i+1}/{len(app_ids)}: {game_data.get('name', 'Unknown')}")
            else:
                print(f"  ❌ {i+1}/{len(app_ids)}: No data for app_id {app_id}")

            time.sleep(delay)  # Be respectful to API

        except Exception as e:
            print(f"  ❌ Error with app_id {app_id}: {e}")
            continue

    print(f"✅ Successfully collected detailed data for {len(detailed_data)} games")
    return detailed_data

# Pull 100 games:
steamspy_data = get_steamspy_detailed_data(max_games=100, delay=1.0)

# Quick preview of the data
if steamspy_data:
    sample_game = list(steamspy_data.values())[0]
    print(f"\n📊 Rich data sample for: {sample_game.get('name', 'Unknown')}")
    print(f"   Average playtime (forever): {sample_game.get('average_forever', 0)} minutes")
    print(f"   Average playtime (2 weeks): {sample_game.get('average_2weeks', 0)} minutes")
    print(f"   Current players: {sample_game.get('ccu', 0):,}")
    print(f"   Positive reviews: {sample_game.get('positive', 0):,}")

🎯 Collecting detailed SteamSpy data for top 100 games...
✅ Retrieved list of 100 top games
📊 Fetching detailed data for 100 games...
  ✅ 1/100: Counter-Strike: Global Offensive
  ✅ 2/100: Apex Legends
  ✅ 3/100: PUBG: BATTLEGROUNDS
  ✅ 4/100: Palworld
  ✅ 5/100: Team Fortress 2
  ✅ 6/100: Call of Duty: Modern Warfare II
  ✅ 7/100: New World: Aeternum
  ✅ 8/100: Black Myth: Wukong
  ✅ 9/100: Grand Theft Auto V Legacy
  ✅ 10/100: Left 4 Dead 2
  ✅ 11/100: Unturned
  ✅ 12/100: Lost Ark
  ✅ 13/100: War Thunder
  ✅ 14/100: Monster Hunter Wilds
  ✅ 15/100: HELLDIVERS 2
  ✅ 16/100: Warframe
  ✅ 17/100: ELDEN RING
  ✅ 18/100: Terraria
  ✅ 19/100: Path of Exile 2
  ✅ 20/100: Wallpaper Engine
  ✅ 21/100: Baldur's Gate 3
  ✅ 22/100: Brawlhalla
  ✅ 23/100: Garry's Mod
  ✅ 24/100: Destiny 2
  ✅ 25/100: Tom Clancy's Rainbow Six Siege
  ✅ 26/100: Rust
  ✅ 27/100: World of Warships
  ✅ 28/100: ARK: Survival Evolved
  ✅ 29/100: Stardew Valley
  ✅ 30/100: The Forest
  ✅ 31/100: War Robots: Frontiers
  ✅

In [ ]:
# Verify Rich SteamSpy Data Collection
print("🔍 VERIFYING RICH STEAMSPY DATA")
print("=" * 50)

if steamspy_data:
    print(f"📊 Total games collected: {len(steamspy_data)}")

    # Check a few games for rich data
    sample_games = list(steamspy_data.items())[:3]

    for app_id, game_data in sample_games:
        print(f"\n🎮 {game_data.get('name', 'Unknown')}:")
        print(f"   Owners: {game_data.get('owners', 'N/A')}")
        print(f"   Average forever: {game_data.get('average_forever', 0):,} minutes")
        print(f"   Average 2weeks: {game_data.get('average_2weeks', 0):,} minutes")
        print(f"   Current players: {game_data.get('ccu', 0):,}")
        print(f"   Positive reviews: {game_data.get('positive', 0):,}")
        print(f"   Negative reviews: {game_data.get('negative', 0):,}")

    print(f"\n✅ Rich engagement data successfully collected!")
else:
    print("❌ No data collected")

🔍 VERIFYING RICH STEAMSPY DATA
📊 Total games collected: 100

🎮 Counter-Strike: Global Offensive:
   Owners: 100,000,000 .. 200,000,000
   Average forever: 32,835 minutes
   Average 2weeks: 861 minutes
   Current players: 1,013,936
   Positive reviews: 7,642,084
   Negative reviews: 1,173,003

🎮 Apex Legends:
   Owners: 100,000,000 .. 200,000,000
   Average forever: 9,859 minutes
   Average 2weeks: 978 minutes
   Current players: 124,262
   Positive reviews: 668,053
   Negative reviews: 326,926

🎮 PUBG: BATTLEGROUNDS:
   Owners: 100,000,000 .. 200,000,000
   Average forever: 22,859 minutes
   Average 2weeks: 1,208 minutes
   Current players: 314,682
   Positive reviews: 1,520,457
   Negative reviews: 1,037,487

✅ Rich engagement data successfully collected!


In [ ]:
# Multi-Source Data Collection - Source 2: Steam Store API
def get_steam_store_data(app_ids, max_games=10, delay=0.5):
    """
    Get detailed game data from Steam Store API
    app_ids: list of Steam app IDs
    max_games: limit for testing (to avoid rate limits)
    delay: seconds to wait between requests
    """
    store_data = {}
    base_url = "https://store.steampowered.com/api/appdetails"

    # Limit to max_games for testing
    limited_ids = app_ids[:max_games]

    print(f"🔍 Fetching Steam Store data for {len(limited_ids)} games...")

    for i, app_id in enumerate(limited_ids):
        try:
            params = {'appids': app_id}
            response = requests.get(base_url, params=params)
            response.raise_for_status()

            data = response.json()

            # Steam Store API returns data with app_id as key
            if app_id in data and data[app_id]['success']:
                store_data[app_id] = data[app_id]['data']
                print(f"  ✅ {i+1}/{len(limited_ids)}: {store_data[app_id].get('name', 'Unknown')}")
            else:
                print(f"  ❌ {i+1}/{len(limited_ids)}: Failed for app_id {app_id}")

            # Rate limiting delay
            time.sleep(delay)

        except Exception as e:
            print(f"  ❌ Error with app_id {app_id}: {e}")
            continue

    print(f"✅ Successfully retrieved {len(store_data)} games from Steam Store API")
    return store_data

# Test with first 5 games from SteamSpy data
app_ids = list(steamspy_data.keys())
steam_store_data = get_steam_store_data(app_ids, max_games=5)

# Preview the data structure
if steam_store_data:
    sample_app_id = list(steam_store_data.keys())[0]
    sample_game = steam_store_data[sample_app_id]

    print(f"\n📊 Sample data structure for: {sample_game.get('name', 'Unknown')}")
    print("Available fields:")
    for key in list(sample_game.keys())[:10]:  # Show first 10 fields
        print(f"  {key}: {sample_game[key]}")





🔍 Fetching Steam Store data for 5 games...
  ✅ 1/5: Counter-Strike 2
  ✅ 2/5: Apex Legends™
  ✅ 3/5: PUBG: BATTLEGROUNDS
  ✅ 4/5: Palworld
  ✅ 5/5: Team Fortress 2
✅ Successfully retrieved 5 games from Steam Store API

📊 Sample data structure for: Counter-Strike 2
Available fields:
  type: game
  name: Counter-Strike 2
  steam_appid: 730
  required_age: 0
  is_free: True
  dlc: [2678630]
  detailed_description: For over two decades, Counter-Strike has offered an elite competitive experience, one shaped by millions of players from across the globe. And now the next chapter in the CS story is about to begin. This is Counter-Strike 2.<br><br>A free upgrade to CS:GO, Counter-Strike 2 marks the largest technical leap in Counter-Strike’s history. Built on the Source 2 engine, Counter-Strike 2 is modernized with realistic physically-based rendering, state of the art networking, and upgraded Community Workshop tools.<br><br>In addition to the classic objective-focused gameplay that Counter-Str

In [ ]:
for key in list(sample_game.keys()):
  print(key)

print(f"Total Fields: {len(list(sample_game.keys()))}")

type
name
steam_appid
required_age
is_free
dlc
detailed_description
about_the_game
short_description
supported_languages
header_image
capsule_image
capsule_imagev5
website
pc_requirements
mac_requirements
linux_requirements
developers
publishers
packages
package_groups
platforms
categories
genres
screenshots
movies
recommendations
achievements
release_date
support_info
background
background_raw
content_descriptors
ratings
Total Fields: 34


### 1.2 Data Quality Assessment & Cleaning

In [ ]:
# Data Quality Assessment
print("🔍 Data Quality Assessment")
print("=" * 50)

# Check SteamSpy data
print(f"\n📊 SteamSpy Data:")
print(f"   Games collected: {len(steamspy_data)}")
print(f"   Sample keys: {list(steamspy_data.keys())[:3]}")

# Check Steam Store data
print(f"\n🏪 Steam Store Data:")
print(f"   Games collected: {len(steam_store_data)}")
print(f"   Sample keys: {list(steam_store_data.keys())[:3]}")

# Check data overlap
common_ids = set(steamspy_data.keys()) & set(steam_store_data.keys())
print(f"\n🔗 Data Overlap:")
print(f"   Games in both datasets: {len(common_ids)}")
print(f"   Missing from Steam Store: {len(steamspy_data) - len(common_ids)}")

# Preview data structure differences
if common_ids:
    sample_id = list(common_ids)[0]
    print(f"\n📋 Data Structure Comparison for App ID {sample_id}:")
    print(f"   SteamSpy fields: {len(steamspy_data[sample_id])}")
    print(f"   Steam Store fields: {len(steam_store_data[sample_id])}")


🔍 Data Quality Assessment

📊 SteamSpy Data:
   Games collected: 100
   Sample keys: ['730', '1172470', '578080']

🏪 Steam Store Data:
   Games collected: 5
   Sample keys: ['730', '1172470', '578080']

🔗 Data Overlap:
   Games in both datasets: 5
   Missing from Steam Store: 95

📋 Data Structure Comparison for App ID 1172470:
   SteamSpy fields: 20
   Steam Store fields: 37


In [ ]:
# Complete Steam Store data collection for all games
print("🔍 Collecting remaining Steam Store data...")

# Get all app IDs we haven't fetched yet
all_app_ids = list(steamspy_data.keys())
already_fetched = set(steam_store_data.keys())
remaining_ids = [app_id for app_id in all_app_ids if app_id not in already_fetched]

print(f"📊 Already have: {len(already_fetched)} games")
print(f"🎯 Need to fetch: {len(remaining_ids)} games")
print(f"⏱️ Estimated time: ~{len(remaining_ids) * 0.5 / 60:.1f} minutes")

if remaining_ids:
    # Fetch remaining games (no max_games limit this time)
    remaining_store_data = get_steam_store_data(remaining_ids, max_games=len(remaining_ids), delay=0.5)

    # Merge with existing data
    steam_store_data.update(remaining_store_data)

    print(f"✅ Total Steam Store games now: {len(steam_store_data)}")

    # Re-run overlap check
    final_overlap = set(steamspy_data.keys()) & set(steam_store_data.keys())
    print(f"🔗 Final overlap: {len(final_overlap)}/{len(steamspy_data)} games")
else:
    print("✅ All games already collected!")


🔍 Collecting remaining Steam Store data...
📊 Already have: 5 games
🎯 Need to fetch: 95 games
⏱️ Estimated time: ~0.8 minutes
🔍 Fetching Steam Store data for 95 games...
  ✅ 1/95: Call of Duty®
  ✅ 2/95: New World: Aeternum
  ✅ 3/95: Black Myth: Wukong
  ✅ 4/95: Grand Theft Auto V Legacy
  ✅ 5/95: Left 4 Dead 2
  ✅ 6/95: Unturned
  ✅ 7/95: Lost Ark
  ✅ 8/95: War Thunder
  ✅ 9/95: Monster Hunter Wilds
  ✅ 10/95: HELLDIVERS™ 2
  ✅ 11/95: Warframe
  ✅ 12/95: ELDEN RING
  ✅ 13/95: Terraria
  ✅ 14/95: Path of Exile 2
  ✅ 15/95: Wallpaper Engine
  ✅ 16/95: Baldur's Gate 3
  ✅ 17/95: Brawlhalla
  ✅ 18/95: Garry's Mod
  ✅ 19/95: Destiny 2
  ✅ 20/95: Tom Clancy's Rainbow Six® Siege X
  ✅ 21/95: Rust
  ✅ 22/95: World of Warships
  ✅ 23/95: ARK: Survival Evolved
  ✅ 24/95: Stardew Valley
  ✅ 25/95: The Forest
  ✅ 26/95: War Robots: Frontiers
  ✅ 27/95: Path of Exile
  ✅ 28/95: PAYDAY 2
  ✅ 29/95: Warface: Clutch
  ✅ 30/95: Grand Theft Auto IV: Complete Edition
  ✅ 31/95: Last Epoch
  ✅ 32/95: Half

In [ ]:
# Final Data Quality Assessment
print("✅ COMPLETE DATA QUALITY ASSESSMENT")
print("=" * 50)

# Final counts
print(f"📊 SteamSpy Data: {len(steamspy_data)} games")
print(f"🏪 Steam Store Data: {len(steam_store_data)} games")
print(f"🔗 Complete overlap: {len(set(steamspy_data.keys()) & set(steam_store_data.keys()))} games")

# Check for missing critical fields
missing_data_issues = []

print(f"\n🔍 Data Quality Issues:")
sample_count = 0
for app_id in list(steam_store_data.keys())[:10]:  # Check first 10 games
    game_data = steam_store_data[app_id]

    # Check for critical missing fields
    if not game_data.get('name'):
        missing_data_issues.append(f"App {app_id}: Missing name")
    if 'is_free' not in game_data:
        missing_data_issues.append(f"App {app_id}: Missing pricing info")
    if not game_data.get('genres'):
        missing_data_issues.append(f"App {app_id}: Missing genres")

    sample_count += 1

if missing_data_issues:
    print(f"❌ Found {len(missing_data_issues)} issues in sample:")
    for issue in missing_data_issues[:5]:  # Show first 5
        print(f"   {issue}")
else:
    print(f"✅ No critical missing data found in sample of {sample_count} games")

print(f"\n🎯 Ready for analysis with {len(steam_store_data)} complete game records!")

✅ COMPLETE DATA QUALITY ASSESSMENT
📊 SteamSpy Data: 100 games
🏪 Steam Store Data: 100 games
🔗 Complete overlap: 100 games

🔍 Data Quality Issues:
✅ No critical missing data found in sample of 10 games

🎯 Ready for analysis with 100 complete game records!


### 1.3 Additional Data Source Integration - Metacritic Scores

Quality scores provide crucial context for understanding the relationship between game quality and commercial success.

In [ ]:
!pip install beautifulsoup4

In [ ]:
import requests
from bs4 import BeautifulSoup
import time

def get_real_metacritic_score(game_name):
    """
    Attempt to get real Metacritic score
    """
    try:
        # Clean game name for URL
        clean_name = game_name.lower()
        clean_name = clean_name.replace(' ', '-').replace(':', '').replace('™', '').replace('®', '')
        clean_name = ''.join(c for c in clean_name if c.isalnum() or c in '-')

        url = f"https://www.metacritic.com/game/pc/{clean_name}/"

        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
        }

        print(f"🔍 Trying: {url}")
        response = requests.get(url, headers=headers, timeout=10)

        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')

            # Look for score (Metacritic uses various selectors)
            score_selectors = [
                'span.metascore_w',
                '.metascore_w',
                '[class*="metascore"]',
                '.c-siteReviewScore_background'
            ]

            for selector in score_selectors:
                score_element = soup.select_one(selector)
                if score_element:
                    score_text = score_element.get_text().strip()
                    if score_text.isdigit():
                        return int(score_text)

            print(f"  📄 Page loaded but no score found")
            return None
        else:
            print(f"  ❌ Status: {response.status_code}")
            return None

    except Exception as e:
        print(f"  ❌ Error: {e}")
        return None

# Test with Counter-Strike 2
test_score = get_real_metacritic_score("Counter-Strike 2")
print(f"Result: {test_score}")

🔍 Trying: https://www.metacritic.com/game/pc/counter-strike-2/
Result: 82


In [ ]:
# Real Metacritic integration for games - test on a subset
def get_real_metacritic_scores(steam_store_data, max_games=20, delay=1.5):
    """
    Get real Metacritic scores for our Steam games
    """
    print(f"🎯 Collecting real Metacritic scores for {max_games} games...")
    print("⚠️  This will take a few minutes to be respectful to their servers")

    metacritic_data = {}

    game_list = list(steam_store_data.items())[:max_games]

    for i, (app_id, game_data) in enumerate(game_list):
        game_name = game_data.get('name', '')
        print(f"\n📊 {i+1}/{max_games}: {game_name}")

        score = get_real_metacritic_score(game_name)

        if score:
            metacritic_data[app_id] = {
                'name': game_name,
                'metacritic_score': score
            }
            print(f"  ✅ Score: {score}")
        else:
            print(f"  ❌ No score found")

        # Be respectful - longer delay between requests
        time.sleep(delay)

    return metacritic_data

# Start with 10 games to test
print("Starting real Metacritic collection...")
metacritic_scores = get_real_metacritic_scores(steam_store_data, max_games=10, delay=1.5)

Starting real Metacritic collection...
🎯 Collecting real Metacritic scores for 10 games...
⚠️  This will take a few minutes to be respectful to their servers

📊 1/10: Counter-Strike 2
🔍 Trying: https://www.metacritic.com/game/pc/counter-strike-2/
  ✅ Score: 82

📊 2/10: Apex Legends™
🔍 Trying: https://www.metacritic.com/game/pc/apex-legends/
  ✅ Score: 89

📊 3/10: PUBG: BATTLEGROUNDS
🔍 Trying: https://www.metacritic.com/game/pc/pubg-battlegrounds/
  ❌ Status: 404
  ❌ No score found

📊 4/10: Palworld
🔍 Trying: https://www.metacritic.com/game/pc/palworld/
  📄 Page loaded but no score found
  ❌ No score found

📊 5/10: Team Fortress 2
🔍 Trying: https://www.metacritic.com/game/pc/team-fortress-2/
  ✅ Score: 92


In [ ]:
# Collect Metacritic scores for all 100 games
print("🎯 FULL METACRITIC COLLECTION - All 100 Games")
print("=" * 60)
print("⚠️  Estimated time: ~3-4 minutes (1.5 sec delay per game)")
print("📊 Starting collection...\n")

# Track our success rate
successful_scores = 0
failed_scores = 0

# Use the function for all games
all_metacritic_scores = get_real_metacritic_scores(steam_store_data, max_games=100, delay=1.5)

# Final summary
print("\n" + "=" * 60)
print("📈 FINAL COLLECTION SUMMARY")
print("=" * 60)
print(f"✅ Successful scores: {len(all_metacritic_scores)}")
print(f"❌ Failed scores: {100 - len(all_metacritic_scores)}")
print(f"📊 Success rate: {len(all_metacritic_scores)/100*100:.1f}%")

if all_metacritic_scores:
    scores = [game['metacritic_score'] for game in all_metacritic_scores.values()]
    print(f"📈 Score range: {min(scores)} - {max(scores)}")
    print(f"📊 Average score: {sum(scores)/len(scores):.1f}")

print(f"\n🎯 Ready for analysis with {len(all_metacritic_scores)} quality scores!")

🎯 FULL METACRITIC COLLECTION - All 100 Games
⚠️  Estimated time: ~3-4 minutes (1.5 sec delay per game)
📊 Starting collection...

🎯 Collecting real Metacritic scores for 100 games...
⚠️  This will take a few minutes to be respectful to their servers

📊 1/100: Counter-Strike 2
🔍 Trying: https://www.metacritic.com/game/pc/counter-strike-2/
  ✅ Score: 82

📊 2/100: Apex Legends™
🔍 Trying: https://www.metacritic.com/game/pc/apex-legends/
  ✅ Score: 89

📊 3/100: PUBG: BATTLEGROUNDS
🔍 Trying: https://www.metacritic.com/game/pc/pubg-battlegrounds/
  ❌ Status: 404
  ❌ No score found

📊 4/100: Palworld
🔍 Trying: https://www.metacritic.com/game/pc/palworld/
  📄 Page loaded but no score found
  ❌ No score found

📊 5/100: Team Fortress 2
🔍 Trying: https://www.metacritic.com/game/pc/team-fortress-2/
  ✅ Score: 92

📊 6/100: Call of Duty®
🔍 Trying: https://www.metacritic.com/game/pc/call-of-duty/
  ✅ Score: 91

📊 7/100: New World: Aeternum
🔍 Trying: https://www.metacritic.com/game/pc/new-world-aeternum

### 1.4 Multi-Source Data Integration & Final Dataset

With 100 Steam games, 100 pricing/metadata records, and 69 quality scores collected, we now combine our three data sources into a unified analytics dataset.

In [ ]:
# Dataset 1: SteamSpy Data
print("📊 1. Preparing SteamSpy Dataset...")
steamspy_records = []

for app_id, data in steamspy_data.items():
    record = {
        'app_id': app_id,
        'name': data.get('name', ''),
        'owners': data.get('owners', ''),
        'average_playtime_forever': data.get('average_forever', 0),
        'average_playtime_2weeks': data.get('average_2weeks', 0),
        'median_playtime_forever': data.get('median_forever', 0),
        'median_playtime_2weeks': data.get('median_2weeks', 0),
        'current_players': data.get('ccu', 0),
        'positive_reviews': data.get('positive', 0),
        'negative_reviews': data.get('negative', 0),
        'user_score': data.get('userscore', 0),
        'developer': data.get('developer', ''),
        'publisher': data.get('publisher', ''),
        'price_current': data.get('price', 0),
        'price_initial': data.get('initialprice', 0),
        'genre': data.get('genre', ''),
        'languages': data.get('languages', '')
    }
    steamspy_records.append(record)

steamspy_df = pd.DataFrame(steamspy_records)
print(f"   ✅ Rich SteamSpy: {len(steamspy_df)} records with engagement data")

# Show the difference - rich data preview
print(f"\n📊 Rich Data Preview:")
print(steamspy_df[['name', 'average_playtime_forever', 'current_players', 'positive_reviews']].head(3))

# Dataset 2: Steam Store Data
print("📊 2. Preparing Steam Store Dataset...")
store_records = []

for app_id, data in steam_store_data.items():
    # Handle genres - extract just the names
    genres_list = []
    if data.get('genres'):
        genres_list = [genre.get('description', '') for genre in data.get('genres', [])]

    # Handle price
    price = 0
    if data.get('price_overview'):
        price = data.get('price_overview', {}).get('final', 0)

    record = {
        'app_id': app_id,
        'name': data.get('name', ''),
        'is_free': data.get('is_free', False),
        'price_cents': price,  # Price in cents
        'required_age': data.get('required_age', 0),
        'supported_languages_count': len(data.get('supported_languages', '').split(',')) if data.get('supported_languages') else 0,
        'genres': ', '.join(genres_list),  # Store as comma-separated string
        'release_date': data.get('release_date', {}).get('date', '') if data.get('release_date') else '',
        'developer': ', '.join(data.get('developers', [])) if data.get('developers') else '',
        'publisher': ', '.join(data.get('publishers', [])) if data.get('publishers') else ''
    }
    store_records.append(record)

store_df = pd.DataFrame(store_records)
print(f"   ✅ Steam Store: {len(store_df)} records")

# Dataset 3: Metacritic Scores
print("📊 3. Preparing Metacritic Dataset...")
metacritic_records = []

for app_id, data in all_metacritic_scores.items():
    record = {
        'app_id': app_id,
        'name': data.get('name', ''),
        'metacritic_score': data.get('metacritic_score', 0)
    }
    metacritic_records.append(record)

metacritic_df = pd.DataFrame(metacritic_records)
print(f"   ✅ Metacritic: {len(metacritic_df)} records")

# Data Quality Summary
print(f"\n📋 DATASET SUMMARY:")
print(f"   SteamSpy games: {len(steamspy_df)}")
print(f"   Steam Store games: {len(store_df)}")
print(f"   Metacritic scores: {len(metacritic_df)}")

# Preview each dataset
print(f"\n👀 Dataset Previews:")
print(f"\nSteamSpy columns: {list(steamspy_df.columns)}")
print(f"Steam Store columns: {list(store_df.columns)}")
print(f"Metacritic columns: {list(metacritic_df.columns)}")

print(f"\n🎯 Ready for BigQuery upload!")

📊 1. Preparing Rich SteamSpy Dataset...
   ✅ Rich SteamSpy: 100 records with engagement data

📊 Rich Data Preview:
                               name  average_playtime_forever  \
0  Counter-Strike: Global Offensive                     32835   
1                      Apex Legends                      9859   
2               PUBG: BATTLEGROUNDS                     22859   

   current_players  positive_reviews  
0          1013936           7642084  
1           124262            668053  
2           314682           1520457  
📊 2. Preparing Steam Store Dataset...
   ✅ Steam Store: 100 records
📊 3. Preparing Metacritic Dataset...
   ✅ Metacritic: 69 records

📋 DATASET SUMMARY:
   SteamSpy games: 100
   Steam Store games: 100
   Metacritic scores: 69

👀 Dataset Previews:

SteamSpy columns: ['app_id', 'name', 'owners', 'average_playtime_forever', 'average_playtime_2weeks', 'median_playtime_forever', 'median_playtime_2weeks', 'current_players', 'positive_reviews', 'negative_reviews', 'user

## 📊 2. BigQuery Analytics Pipeline

### 2.1 Dataset Upload to Cloud Data Warehouse

In [ ]:
# Fix Data Types Before BigQuery Upload
print("🔧 FIXING DATA TYPES FOR BIGQUERY")
print("=" * 50)

# Fix Rich SteamSpy DataFrame
print("📊 Cleaning Rich SteamSpy data types...")
steamspy_df['app_id'] = steamspy_df['app_id'].astype(str)
steamspy_df['average_playtime_forever'] = pd.to_numeric(steamspy_df['average_playtime_forever'], errors='coerce').fillna(0).astype(int)
steamspy_df['average_playtime_2weeks'] = pd.to_numeric(steamspy_df['average_playtime_2weeks'], errors='coerce').fillna(0).astype(int)
steamspy_df['median_playtime_forever'] = pd.to_numeric(steamspy_df['median_playtime_forever'], errors='coerce').fillna(0).astype(int)
steamspy_df['median_playtime_2weeks'] = pd.to_numeric(steamspy_df['median_playtime_2weeks'], errors='coerce').fillna(0).astype(int)
steamspy_df['current_players'] = pd.to_numeric(steamspy_df['current_players'], errors='coerce').fillna(0).astype(int)
steamspy_df['positive_reviews'] = pd.to_numeric(steamspy_df['positive_reviews'], errors='coerce').fillna(0).astype(int)
steamspy_df['negative_reviews'] = pd.to_numeric(steamspy_df['negative_reviews'], errors='coerce').fillna(0).astype(int)
steamspy_df['user_score'] = pd.to_numeric(steamspy_df['user_score'], errors='coerce').fillna(0).astype(int)
steamspy_df['price_current'] = pd.to_numeric(steamspy_df['price_current'], errors='coerce').fillna(0).astype(int)
steamspy_df['price_initial'] = pd.to_numeric(steamspy_df['price_initial'], errors='coerce').fillna(0).astype(int)

# Fix Steam Store DataFrame
print("🏪 Cleaning Steam Store data types...")
store_df['app_id'] = store_df['app_id'].astype(str)
store_df['is_free'] = store_df['is_free'].astype(bool)
store_df['price_cents'] = pd.to_numeric(store_df['price_cents'], errors='coerce').fillna(0).astype(int)
store_df['required_age'] = pd.to_numeric(store_df['required_age'], errors='coerce').fillna(0).astype(int)
store_df['supported_languages_count'] = pd.to_numeric(store_df['supported_languages_count'], errors='coerce').fillna(0).astype(int)

# Fix Metacritic DataFrame
print("🎯 Cleaning Metacritic data types...")
metacritic_df['app_id'] = metacritic_df['app_id'].astype(str)
metacritic_df['metacritic_score'] = pd.to_numeric(metacritic_df['metacritic_score'], errors='coerce').fillna(0).astype(int)

print("✅ Data types cleaned!")

# Check data types
print(f"\nSteam Store dtypes:")
print(store_df.dtypes)

🔧 FIXING DATA TYPES FOR BIGQUERY
📊 Cleaning Rich SteamSpy data types...
🏪 Cleaning Steam Store data types...
🎯 Cleaning Metacritic data types...
✅ Data types cleaned!

Steam Store dtypes:
app_id                       object
name                         object
is_free                        bool
price_cents                   int64
required_age                  int64
supported_languages_count     int64
genres                       object
release_date                 object
developer                    object
publisher                    object
dtype: object


In [ ]:
# Upload Rich Datasets to BigQuery
print("📤 RE-UPLOADING RICH DATASETS TO BIGQUERY")
print("=" * 60)

# Upload Table 1: Rich SteamSpy Data (UPDATED)
print(f"\n📊 Uploading Rich SteamSpy data...")
table_id_1 = f"{PROJECT_ID}.steam_analytics.steamspy_data"
job_config_1 = bigquery.LoadJobConfig(write_disposition="WRITE_TRUNCATE")

job_1 = client.load_table_from_dataframe(steamspy_df, table_id_1, job_config=job_config_1)
job_1.result()  # Wait for job to complete
print(f"✅ Uploaded {len(steamspy_df)} rows with rich engagement data")

# Upload Table 2: Steam Store Data (same as before)
print(f"\n🏪 Re-uploading Steam Store data...")
table_id_2 = f"{PROJECT_ID}.steam_analytics.steam_store_data"
job_config_2 = bigquery.LoadJobConfig(write_disposition="WRITE_TRUNCATE")

job_2 = client.load_table_from_dataframe(store_df, table_id_2, job_config=job_config_2)
job_2.result()
print(f"✅ Uploaded {len(store_df)} rows to steam_store_data table")

# Upload Table 3: Metacritic Scores (same as before)
print(f"\n🎯 Re-uploading Metacritic data...")
table_id_3 = f"{PROJECT_ID}.steam_analytics.metacritic_scores"
job_config_3 = bigquery.LoadJobConfig(write_disposition="WRITE_TRUNCATE")

job_3 = client.load_table_from_dataframe(metacritic_df, table_id_3, job_config=job_config_3)
job_3.result()
print(f"✅ Uploaded {len(metacritic_df)} rows to metacritic_scores table")

print(f"\n🎯 SUCCESS! Rich dataset uploaded to BigQuery!")
print(f"📍 Ready for advanced SQL analysis!")

📤 RE-UPLOADING RICH DATASETS TO BIGQUERY

📊 Uploading Rich SteamSpy data...
✅ Uploaded 100 rows with rich engagement data

🏪 Re-uploading Steam Store data...
✅ Uploaded 100 rows to steam_store_data table

🎯 Re-uploading Metacritic data...
✅ Uploaded 69 rows to metacritic_scores table

🎯 SUCCESS! Rich dataset uploaded to BigQuery!
📍 Ready for advanced SQL analysis!
